In [1]:
!pip install folium
import folium
from folium import Marker
from folium import plugins
from folium import GeoJson
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go


   ---------------------------------------- 0.0/100.0 kB ? eta -:--:--
   ---------------------------------------- 100.0/100.0 kB ? eta 0:00:00
   ---------------------------------------- 0.0/133.2 kB ? eta -:--:--
   ---------------------------------------- 133.2/133.2 kB 7.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/62.6 kB ? eta -:--:--
   ---------------------------------------- 62.6/62.6 kB 3.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/56.3 kB ? eta -:--:--
   ---------------------------------------- 56.3/56.3 kB 2.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/163.8 kB ? eta -:--:--
   ---------------------------------------- 163.8/163.8 kB 9.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/99.6 kB ? eta -:--:--
   ---------------------------------------- 99.6/99.6 kB ? eta 0:00:00
   ---------------------------------------- 0.0/61.6 kB ? eta -:--:--
   ---------------------------------------- 61.

In [2]:
# 서울 스타벅스 지점 데이터 불러오기
df = pd.read_csv('../datasets/Starbucks_Seoul.csv')

# 지역 구분을 위한 json 파일 불러오기
geo = '../datasets/Seoul_Gu.json'

df.head()

,name,address,gu_name,latitude,longitude
0,GS타워,서울특별시 강남구 논현로 508 (역삼동),강남구,37.501859,127.037278
1,SSG마켓도곡R,"서울특별시 강남구 언주로30길 57, 타워팰리스Ⅱ F 지하1층 (도곡동)",강남구,37.490298,127.054895
2,W-Mall,서울특별시 금천구 디지털로 188 (가산동),금천구,37.477305,126.887691
3,가든파이브,서울특별시 송파구 충민로 10 (문정동) 가든파이브툴,송파구,37.478232,127.119370
4,가락본동,서울특별시 송파구 송파대로30길 13 (가락동),송파구,37.494895,127.118785


In [4]:
# 기본 지도 시각화(서울의 위도, 경도 입력)

m = folium.Map(location=[37.541, 126.986], zoom_start=12)
m

In [8]:
# 지도 형태 변경
m = folium.Map(location=[37.541, 126.986], titles='Stamen Terrain', zoom_start=12)

# 원하는 지표에 반경(radius) 표시(남산)
folium.CircleMarker([37.5538, 126.9810], radius=50, 
                    popup='Laurelhurst Park', color='#3246cc', 
                    fill_color='#3246cc').add_to(m)

# 원하는 지표에 포인트 표시(남산)
folium.Marker([37.5538, 126.9810], popup='The Waterfront').add_to(m)

m

In [10]:
# 서울 지도에 스타벅스 지점 수 시각화

m = folium.Map([37.541, 126.986], zoom_start=12, width='100%', height='100%')
locations = list(zip(df.latitude, df.longitude))
cluster = plugins.MarkerCluster(locations=locations, popups=df['name'].tolist())

m.add_child(cluster)
m

In [11]:
# 서울 지도에 스타벅스 지점 수 도트맵 시각화

m = folium.Map(location=[37.541, 126.986], zoom_start=12, width='100%', height='100%')
locations = list(zip(df.latitude, df.longitude))
for i in range(len(locations)):
    folium.CircleMarker(location=locations[i], radius=1).add_to(m)

m

In [12]:
# 서울 구별 스타벅스 지점 수 집계 및 중심점 산출
df_m = df.groupby('gu_name').agg({'latitude':'mean', 'longitude':'mean',
                                  'name':'count'}).reset_index()
df_m.head()


,gu_name,latitude,longitude,name
0,강남구,37.507603,127.044611,80
1,강동구,37.539914,127.137106,14
2,강북구,37.626866,127.026372,5
3,강서구,37.555716,126.841528,16
4,관악구,37.481759,126.944286,11


In [14]:
# 서울 구별 스타벅스 지점 수 버블맵 시각화

# 기본 지도 생성
m = folium.Map(location=[37.541, 126.986], tiles='Cartodb Positron', zoom_start=11,
               width='100%', height='100%')

# 구별 구분선, 색상 설정
folium.Choropleth(geo_data=geo, # 앞에서 불러온 geo 파일 적용
                  fill_color='gray').add_to(m)

# 버블맵 삽입
locations = list(zip(df_m.latitude, df_m.longitude))
for i in range(len(locations)):
    row = df_m.iloc[i]
    folium.CircleMarker(location=locations[i],
                        radius=float(row.name/2),
                        fill_color='blue').add_to(m)

m

In [19]:
# 미국 실업률 정보의 코로플레스맵 시각화를 위한 데이터, json 불러오기
df2 = pd.read_csv('../datasets/us_states_unemployment.csv')

# 주별 경계 json file 불러오기
us_geo = '../datasets/folium_us-states.json'

df2.head()

,State,Unemployment
0,AL,7.1
1,AK,6.8
2,AZ,8.1
3,AR,7.2
4,CA,10.1


In [21]:
# 미국 주별 실업률 코로플레스맵 시각화

# 미국 지도 시각화
m = folium.Map(location=[40, -98], zoom_start=3, tiles='Cartodb Positron')

# 지도에 주 경계선, 실업률 데이터 연동
folium.Choropleth(geo_data =us_geo,
             data=df2,
             columns=['State', 'Unemployment'], 
             key_on = 'feature.id', # json과 실업률 데이터를 연결할 키값 설정
             fill_color = 'YlGn',
             legend_name='실업률').add_to(m)

m

In [22]:
# 서울과 각국의 수도 간의 커넥션맵 시각화

# 서울과 도쿄, 워싱턴, 마닐라, 파리, 모스크바 위경도 입력
source_to_dest = zip([37.541,37.541,37.541,37.541,37.541], 
                     [35.6804, 38.9072, 14.5995, 48.8566,55.7558],
                     [126.986,126.986,126.986,126.986,126.986], 
                     [139.7690, -77.0369, 120.9842, 2.3522,37.6173])

fig = go.Figure()

# for문을 활용하여 위경도 입력
for a, b, c, d in source_to_dest:
    fig.add_trace(go.Scattergeo(
        lat=[a,b],
        lon=[c,d],
        mode='lines',
        line=dict(width=1, color='red'),
        opacity=0.5
    ))

fig.update_layout(margin={'t':0, "b":0, "l":0, "r":0, "pad":0},
                  showlegend=False, geo=dict(showcountries=True))

fig.show()